In [7]:
from urllib.request import urlopen
from urllib import parse
from selenium import webdriver
from bs4 import BeautifulSoup

from flask import Flask
from flask import request
from flask import jsonify

import time
import os
import json
import re
import sys, traceback
import threading

KW_CODE =0
KM_CODE =1
DJ_CODE=2
DS_CODE=3
DD_CODE=4
MJ_CODE=5
SY_CODE=6
SM_CODE=7
SK_CODE=8
SW_CODE=9
SS_CODE=10
HS_CODE=11


# In[4]:


base_start_url_list = ['http://kupis.kw.ac.kr/',
'http://lib.kookmin.ac.kr/',
'http://library.daejin.ac.kr/',
'http://discover.duksung.ac.kr/',
'http://library.dongduk.ac.kr/',
'http://lib.mju.ac.kr/',
'http://lib.syu.ac.kr/',
'https://lib.smu.ac.kr/',
'http://library.skuniv.ac.kr/',
'http://lib.swu.ac.kr/',
'http://lib.sungshin.ac.kr/',
'http://hsel.hansung.ac.kr/']

base_middle_url_list = ['search/caz/result?st=KWRD&commandType=advanced&si=1&q=',
'#/search/ex?isbn=1%7Cl%7Ca%7C',
'data_data_list.mir?search_keyword_type1=isbn&search_keyword1=',
'#/search/ex?isbn=1%7Ck%7Ca%7C',
'#/search/ex?isbn=1%7Cl%7Ca%7C',
'search/Search.Result.ax',
'',
'Search/?q=',
'',
'',
'',
'']





base_end_url_list = ['&b0=and&weight0=&si=2&q=&b1=and&weight1=&si=3&q=&weight2=&inc=TOTAL&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&lmt0=TOTAL&lmtsn=000000000003&lmtst=OR&rf=&rt=&range=000000000021&cpp=10&msc=100',
'&branch=1&material-type=1',
'&srch_condi_01=AND&search_keyword_type2=author&search_keyword2=&srch_condi_02=AND&search_keyword_type3=subject&search_keyword3=&srch_condi_03=%E2%96%B2&select_current_sort_type=&select_current_sort_order=&select_current_per_page_num=10&basic_list=list&current_per_page_num=10&current_sort_type=&current_sort_order=&MAXI=1000&mloc_code=DJUL&facet_limit_search_field_code=&scope_code=&mtype_code=M&type_code=',
'&material-type=1',
'&branch=1&material-type=1',
'?sid=1&q=ISBN%3A{}%3A1&eq=&mf=true&qt=ISBN%3D{}&qf={}&f=&br=&cl=&gr=1+3+4+5+6+7+8+9+10+11+12+13+14+2+15+16+26+27+17+18+19+20+21+22+23+24+25+28+29+30+33+34&rl=&page=&pageSize=0&s=&st=&h=&cr=&py=&subj=&facet=Y&nd=&vid=0&fv=%24esc.html%28%24%21%7BFilters%7D%29&tabID=',
'',
'',
'',
'',
'',
''
]



class LibraryLS:
    def __init__(self, sid):
        self.sid = sid
        self.loanStatusList = []
        self.errorMessage= ''
    


# In[8]:


class LoanStatus:
    def __init__(self, title = '', RN = '', CN = '', POS = '', STATE = -1, RDD = '', BN = 0, errorMessage = ''):
        self.title = title
        self.RN= RN
        self.CN= CN
        self.POS= POS
        self.STATE= STATE
        self.RDD = RDD#'2222-12-31'
        self.BN = BN
        self.errorMessage = errorMessage
'''#STATE : 도서 상태,
#-1  에러
#1 대출가능
#0 대출중
#2 지정도서
# 대출불가 아직 생각 안함.
#RN : 등록번호
#CN : 청구기호
#POS : 위치
이런것들'''



stateDict = {'대출중': 0, '예약도서대출중':0, '폐가도서대출중':0,'행낭대출중':0, '이용가능':1,'대출가능':1, '지정도서':2}



In [1]:
#https://lib.mju.ac.kr/search/Search.Result.ax?sid=1&q=ISBN%3A9788996094050%3A1&eq=&mf=true&qt=ISBN%3D9788996094050&qf=9788996094050&f=&br=&cl=&gr=1+3+4+5+6+7+8+9+10+11+12+13+14+2+15+16+26+27+17+18+19+20+21+22+23+24+25+28+29+30+33+34&rl=&page=&pageSize=0&s=&st=&h=&cr=&py=&subj=&facet=Y&nd=&vid=0&fv=%24esc.html%28%24%21%7BFilters%7D%29&tabID=

In [11]:
def makeSearchUrl(ISBN, title, sid, flag):
    if flag ==1:
        if sid == MJ_CODE :
            searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + base_end_url_list[sid].format(ISBN, ISBN, ISBN)
        else :
            searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + ISBN+ base_end_url_list[sid]
    else :
        # flag 2.
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + title+ base_end_url_list[sid]
    return searchUrl

In [3]:
testStr =  ''

In [5]:
testStr.format('9788996094050', '9788996094050', '9788996094050')

'?sid=1&q=ISBN%3A9788996094050%3A1&eq=&mf=true&qt=ISBN%3D9788996094050&qf=9788996094050&f=&br=&cl=&gr=1+3+4+5+6+7+8+9+10+11+12+13+14+2+15+16+26+27+17+18+19+20+21+22+23+24+25+28+29+30+33+34&rl=&page=&pageSize=0&s=&st=&h=&cr=&py=&subj=&facet=Y&nd=&vid=0&fv=%24esc.html%28%24%21%7BFilters%7D%29&tabID='

In [13]:
makeSearchUrl('9788996094050','(윤성우의)열혈 C 프로그래밍', MJ_CODE, 1)

'http://lib.mju.ac.kr/search/Search.Result.ax?sid=1&q=ISBN%3A9788996094050%3A1&eq=&mf=true&qt=ISBN%3D9788996094050&qf=9788996094050&f=&br=&cl=&gr=1+3+4+5+6+7+8+9+10+11+12+13+14+2+15+16+26+27+17+18+19+20+21+22+23+24+25+28+29+30+33+34&rl=&page=&pageSize=0&s=&st=&h=&cr=&py=&subj=&facet=Y&nd=&vid=0&fv=%24esc.html%28%24%21%7BFilters%7D%29&tabID='

In [14]:
makeSearchUrl('9791186757093','자존감수업', MJ_CODE, 1)

'http://lib.mju.ac.kr/search/Search.Result.ax?sid=1&q=ISBN%3A9791186757093%3A1&eq=&mf=true&qt=ISBN%3D9791186757093&qf=9791186757093&f=&br=&cl=&gr=1+3+4+5+6+7+8+9+10+11+12+13+14+2+15+16+26+27+17+18+19+20+21+22+23+24+25+28+29+30+33+34&rl=&page=&pageSize=0&s=&st=&h=&cr=&py=&subj=&facet=Y&nd=&vid=0&fv=%24esc.html%28%24%21%7BFilters%7D%29&tabID='

In [65]:
import re

import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import copy


options = Options()
options.headless = True

browser = webdriver.Chrome(executable_path="./chromedriver.exe", options=options)

#browser.get('https://library.dongduk.ac.kr/#/search/ex?isbn=1%7Cl%7Ca%7C9791165075064&branch=1&material-type=1')
# 되는 케이스

#browser.get('http://lib.mju.ac.kr/search/Search.Result.ax?sid=1&q=ISBN%3A9788996094050%3A1&eq=&mf=true&qt=ISBN%3D9788996094050&qf=9788996094050&f=&br=&cl=&gr=1+3+4+5+6+7+8+9+10+11+12+13+14+2+15+16+26+27+17+18+19+20+21+22+23+24+25+28+29+30+33+34&rl=&page=&pageSize=0&s=&st=&h=&cr=&py=&subj=&facet=Y&nd=&vid=0&fv=%24esc.html%28%24%21%7BFilters%7D%29&tabID=')
#9788931004441 새로운 케이스


#browser.get(makeSearchUrl('9791186757093','자존감수업', MJ_CODE, 1))
browser.get(makeSearchUrl('9791186757000','없는책', MJ_CODE, 1))
#대출중 있는 것



#9788953934283 새로운 케이스
#browser.get('https://library.daejin.ac.kr/data_data_list.mir?search_keyword_type1=isbn&search_keyword1=9788953934283&srch_condi_01=AND&search_keyword_type2=author&search_keyword2=&srch_condi_02=AND&search_keyword_type3=subject&search_keyword3=&srch_condi_03=%E2%96%B2&select_current_sort_type=&select_current_sort_order=&select_current_per_page_num=10&basic_list=list&current_per_page_num=10&current_sort_type=&current_sort_order=&MAXI=1000&mloc_code=DJUL&facet_limit_search_field_code=&scope_code=&mtype_code=M&type_code=')
# 되는 케이스

#browser.get('https://discover.duksung.ac.kr/#/search/ex?isbn=1%7Ck%7Ca%7C9788934985099&material-type=1')
#안되는 케이스

In [70]:
browser.current_url

'https://lib.mju.ac.kr/search/Search.Result.ax?sid=1&q=ISBN%3A9791186757000%3A1&eq=&mf=true&qt=ISBN%3D9791186757000&qf=9791186757000&f=&br=&cl=&gr=1+3+4+5+6+7+8+9+10+11+12+13+14+2+15+16+26+27+17+18+19+20+21+22+23+24+25+28+29+30+33+34&rl=&page=&pageSize=0&s=&st=&h=&cr=&py=&subj=&facet=Y&nd=&vid=0&fv=%24esc.html%28%24%21%7BFilters%7D%29&tabID='

In [77]:
linkXpath = '//*[@id="frmResultList"]/ul/li/dl/dd/div/div/div[3]/a[1]'
#$(SELECTOR)
browserCheckXpath = '//*[@id="container_inner"]/div[1]/h2/img'
#소장검색 이라는 단어

try:
    WebDriverWait(browser, 120).until(EC.presence_of_element_located((By.XPATH, browserCheckXpath)))
    linkElement = WebDriverWait(browser, 2).until(EC.presence_of_element_located((By.XPATH, linkXpath)))
    #linkElement = browser.find_element_by_xpath(linkXpath)
    # 오류가능성.
    if linkElement.text == '':
        linkElement = -1
except :
    print('예외가 발생! 찾는 책이 없음')
    linkElement = -1

In [78]:
linkElement.text

AttributeError: 'int' object has no attribute 'text'

In [79]:
if linkElement == -1 :
    print('-1...')
else : 
    print('success')

-1...


In [29]:
linkElement.click()

In [30]:
browser.current_url

'https://lib.mju.ac.kr/search/DetailView.ax?sid=1&cid=3794834'

In [32]:
browserCheckXpath = '//*[@id="container_inner"]/div[1]/h2/img'
#소장검색 이라는 단어

try:
    WebDriverWait(browser, 120).until(EC.presence_of_element_located((By.XPATH, browserCheckXpath)))
except :
    print('error detected')

time.sleep(1)
html = browser.page_source
bsObject=BeautifulSoup(html, 'html.parser')
strISBNIncluded = str(bsObject.select('#detailMetaData'))

In [33]:
strISBNIncluded

'[<td id="detailMetaData" style="padding-top:10px;">\n<div id="MetaData">\n<table width="100%">\n<tbody><tr>\n<td>\n<table width="100%">\n<tbody id="metaDataBody">\n<tr>\n<td class="detailHead">자료유형 :</td>\n<td class="detailBody">동양서단행본</td>\n</tr>\n<tr>\n<td class="detailHead">서명 / 저자 : </td><td class="detailBody">자존감 수업 : 하루에 하나, 나를 사랑하게 되는 자존감 회복 훈련 / 윤홍균 지음.</td>\n</tr><tr>\n<td class="detailHead">기타서명 : </td><td class="detailBody">오늘부터 자존감 있는 삶!.<br/></td>\n</tr><tr>\n<td class="detailHead">개인저자 : </td><td class="detailBody"> <a href="javascript:search.searchLink(\'AUTH\',\'윤홍균. \');">윤홍균. </a></td>\n</tr><tr>\n<td class="detailHead">발행사항 : </td><td class="detailBody">[서울] : 심플라이프, 2016. </td>\n</tr><tr>\n<td class="detailHead">형태사항 : </td><td class="detailBody">304 p. : 삽도 ; 23 cm.</td>\n</tr><tr>\n<td class="detailHead">키워드 : </td><td class="detailBody">\n<a href="javascript:search.searchLink(\'ALL\',\'자존감\');">자존감</a></td>\n</tr><tr>\n<td class="detailHead">ISBN : </td><td clas

In [34]:
def compareISBN(text, ISBN):
    regex = re.compile(ISBN)
    matchObj = regex.search(text)
    if matchObj is None :
        return False
    else:
        return True

In [58]:
print(compareISBN(strISBNIncluded, '9791186757093'))
#진짜
print(compareISBN(strISBNIncluded, '9791165075062'))
#가짜.

True
False


In [59]:
num = 1
#itmdtl_info01 > div > table > tbody > tr
loanResultList = []
while True:
    semiLoanResultList = bsObject.select('#itmdtl_info0'+str(num)+' > div > table > tbody > tr')
    if len(semiLoanResultList) == 0 :
        break
    else :
        loanResultList.extend(semiLoanResultList)
    num +=1

len(loanResultList)        
# loanResultList =bsObject.select('#itmdtl_info01 > div > table > tbody > tr')
# len(loanResultList)
# loanResultList =bsObject.select('#itmdtl_info02 > div > table > tbody > tr')
# len(loanResultList)

# loanResultList =bsObject.select('#itmdtl_info03 > div > table > tbody > tr')
# type(loanResultList)

7

In [62]:
def findState(text, sid = -1):
    if sid == KM_CODE or sid == DS_CODE or sid == DD_CODE:
        regex = re.compile('(\w+)\s+')
        matchObj = regex.search(text)
        text = matchObj.group(1)
    # 국민대 의 경우 전처리.

    try:
        STATE = stateDict[text]
    except KeyError:
        STATE = -1
    return STATE

In [64]:
for item in loanResultList:
    RDD = ''
    content = item.find_all('td')
    print(len(content))
    errorMessage = ''
    RN = content[1].text.strip()
    CN = content[3].text.strip()
    POS = content[2].text.strip()
    STATE_STRING = content[4].text.strip()
    STATE = findState(STATE_STRING, MJ_CODE)
    if STATE == 0 :
        RDD  =content[5].text.strip()
        
    #RDD = content[5].text.strip()
    print(RN)
    print(CN)
    print(POS)
    print(STATE_STRING, STATE)
    print(RDD)
    #print(content[8])
        
    #print('등록번호 : ', item)

7
7001326651
155.2 윤95자
4층인문과학자료실
대출가능 1

7
7001324589
155.2 윤95자
4층인문과학자료실
대출가능 1

7
7001332393
155.2 윤95자
4층인문과학자료실
대출중 0
2020-06-26
7
3000685932
155.2 윤95자
3층인문예술단행본
대출가능 1

7
3000677585
155.2 윤95자
3층인문예술단행본
대출가능 1

7
3000677583
155.2 윤95자
3층인문예술단행본
대출가능 1

7
3000685933
155.2 윤95자
3층인문예술단행본
대출중 0
2019-08-27


In [55]:
li = [1,2,3]
li2 = [4,5,6]
li3 =li.extend(li2)
print(li3)

None


In [56]:
li

[1, 2, 3, 4, 5, 6]